In [3]:
import nltk
#for X, Y
# def data_preparing(filename):
#     f = open(filename)
#     X = []
#     X_sentence = []
#     Y_sentence = []
#     Y = []
#     for line in f:
#         if len(line)!=1:
#             split = line.split()
#             X_sentence.append([split[0]])
#             Y_sentence.append(split[1])
#         else:
#             tmp = []
#             for i in range(0, len(X_sentence)):
#                 tmp.append(X_sentence[i][0])
#             pos = nltk.pos_tag(tmp)
#             for j in range (0, len(X_sentence)):
#                 X_sentence[j].insert(1, pos[j][1])
#             X.append(X_sentence)
#             Y.append(Y_sentence)
#             X_sentence = []
#             Y_sentence = []
#     return X, Y

# X_train, Y_train = data_preparing("sequence_labelling_aspects/train_aspect.tsv")
# X_test, Y_test = data_preparing("sequence_labelling_aspects/test_aspect.tsv")
# X_dev, Y_dev = data_preparing("sequence_labelling_aspects/dev_aspect.tsv")

#for sentence
def data_preparing(filename):
    f = open(filename)
    sentences = []
    sentence = []
    for line in f:
        if len(line)!=1:
            sentence.append(line.split())
        else:
            tmp = []
            for i in range(len(sentence)):
                tmp.append(sentence[i][0])
            pos = nltk.pos_tag(tmp)
            for j in range (len(sentence)):
                sentence[j].insert(1, pos[j][1])
            sentences.append(sentence)
            sentence = []
    return sentences

train_sentences = data_preparing("sequence_labelling_predicate/train_pred_full.tsv")
test_sentences = data_preparing("sequence_labelling_predicate/test_pred_full.tsv")
dev_sentences = data_preparing("sequence_labelling_predicate/dev_pred_full.tsv")
train_sentences[0]

[['also', 'RB', 'O'],
 [',', ',', 'O'],
 ['i', 'NNS', 'O'],
 ['have', 'VBP', 'O'],
 ['recently', 'RB', 'O'],
 ['discovered', 'VBN', 'O'],
 ['advil', 'NN', 'B-OBJ'],
 ['liquigels', 'NNS', 'O'],
 ['work', 'VBP', 'O'],
 ['much', 'RB', 'O'],
 ['better', 'JJR', 'O'],
 ['and', 'CC', 'O'],
 ['faster', 'RBR', 'B-PREDFULL'],
 ['for', 'IN', 'I-PREDFULL'],
 ['a', 'DT', 'I-PREDFULL'],
 ['headache', 'NN', 'I-PREDFULL'],
 ['than', 'IN', 'O'],
 ['regular', 'JJ', 'O'],
 ['ibuprofen', 'NN', 'B-OBJ'],
 ['.', '.', 'O']]

In [4]:
len(train_sentences)

3077

In [5]:
len(test_sentences)

488

In [6]:
len(dev_sentences)

402

In [7]:
def word2features(sent, i):
    word = sent[i][0]
    postag = sent[i][1]

    features = {
        'bias': 1.0,
        'word.lower()': word.lower(),
        'word[-3:]': word[-3:],
        'word[-2:]': word[-2:],
        'word.isupper()': word.isupper(),
        'word.istitle()': word.istitle(),
        'word.isdigit()': word.isdigit(),
        'postag': postag,
        'postag[:2]': postag[:2],
    }
    if i > 0:
        word1 = sent[i-1][0]
        postag1 = sent[i-1][1]
        features.update({
            '-1:word.lower()': word1.lower(),
            '-1:word.istitle()': word1.istitle(),
            '-1:word.isupper()': word1.isupper(),
            '-1:postag': postag1,
            '-1:postag[:2]': postag1[:2],
        })
    else:
        features['BOS'] = True

    if i < len(sent)-1:
        word1 = sent[i+1][0]
        postag1 = sent[i+1][1]
        features.update({
            '+1:word.lower()': word1.lower(),
            '+1:word.istitle()': word1.istitle(),
            '+1:word.isupper()': word1.isupper(),
            '+1:postag': postag1,
            '+1:postag[:2]': postag1[:2],
        })
    else:
        features['EOS'] = True

    return features


def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

def sent2labels(sent):
    return [label for token, postag, label in sent]

def sent2tokens(sent):
    return [token for token, postag, label in sent]

In [8]:
X_train = [sent2features(s) for s in train_sentences]
y_train = [sent2labels(s) for s in train_sentences]

X_test = [sent2features(s) for s in test_sentences]
y_test = [sent2labels(s) for s in test_sentences]

X_dev = [sent2features(s) for s in dev_sentences]
y_dev = [sent2labels(s) for s in dev_sentences]
y_dev[6]

['B-OBJ',
 'I-OBJ',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B-PREDFULL',
 'O',
 'B-OBJ',
 'I-OBJ',
 'O',
 'O',
 'O',
 'O',
 'O']

In [9]:
import sklearn
import sklearn_crfsuite
from sklearn_crfsuite import scorers
crf = sklearn_crfsuite.CRF(algorithm='l2sgd' , all_possible_transitions=True)
crf.fit(X_train, y_train)

CRF(algorithm='l2sgd', all_possible_states=None, all_possible_transitions=True,
    averaging=None, c=None, c1=None, c2=None, calibration_candidates=None,
    calibration_eta=None, calibration_max_trials=None, calibration_rate=None,
    calibration_samples=None, delta=None, epsilon=None, error_sensitive=None,
    gamma=None, keep_tempfiles=None, linesearch=None, max_iterations=None,
    max_linesearch=None, min_freq=None, model_filename=None, num_memories=None,
    pa_type=None, period=None, trainer_cls=None, variance=None, verbose=False)

In [17]:
preds = crf.predict(X_test)
test_labels = y_test

In [18]:
from seqeval.metrics import precision_score, recall_score, f1_score, classification_report

print(f'Precision:', precision_score(test_labels, preds))
print(f'Recall:', recall_score(test_labels, preds))
print(f'Entity-level f1: {str(f1_score(test_labels, preds))}')
print(f'Report:\n {str(classification_report(test_labels, preds))}')

Precision: 0.579250720461095
Recall: 0.16003184713375795
Entity-level f1: 0.2507797878976918
Report:
            precision    recall  f1-score   support

      OBJ       0.55      0.12      0.20      1092
 PREDFULL       0.66      0.41      0.50       164

micro avg       0.58      0.16      0.25      1256
macro avg       0.56      0.16      0.24      1256



In [21]:
labels = list(crf.classes_)
labels.remove('O')
labels

['B-OBJ', 'B-PREDFULL', 'I-PREDFULL', 'I-OBJ']

In [22]:
from sklearn_crfsuite import metrics
y_pred = crf.predict(X_dev)
metrics.flat_f1_score(y_dev, y_pred,
                      average='weighted', labels=labels)

0.08760407754569967

In [23]:
sorted_labels = sorted(
    labels,
    key=lambda name: (name[1:], name[0])
)
print(metrics.flat_classification_report(
    y_dev, y_pred, labels=sorted_labels, digits=3
))

              precision    recall  f1-score   support

       B-OBJ      0.000     0.000     0.000      1097
       I-OBJ      0.000     0.000     0.000         2
  B-PREDFULL      0.644     0.475     0.546       118
  I-PREDFULL      0.697     0.622     0.657        74

   micro avg      0.537     0.079     0.138      1291
   macro avg      0.335     0.274     0.301      1291
weighted avg      0.099     0.079     0.088      1291



In [33]:
import scipy.stats
from sklearn.metrics import make_scorer
from sklearn.model_selection import RandomizedSearchCV

%time
# define fixed parameters and parameters to search
crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    max_iterations=100,
    all_possible_transitions=True
)
params_space = {
    'c1': scipy.stats.expon(scale=0.5),
    'c2': scipy.stats.expon(scale=0.05),
}

# use the same metric for evaluation
f1_scorer = make_scorer(metrics.flat_f1_score,
                        average='weighted', labels=labels)

# search
rs = RandomizedSearchCV(crf, params_space,
                        cv=3,
                        verbose=1,
                        n_jobs=-1,
                        n_iter=50,
                        scoring=f1_scorer)
rs.fit(X_train, y_train)

Wall time: 0 ns
Fitting 3 folds for each of 50 candidates, totalling 150 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:  2.6min
[Parallel(n_jobs=-1)]: Done 150 out of 150 | elapsed: 14.3min finished


RandomizedSearchCV(cv=3, error_score='raise-deprecating',
                   estimator=CRF(algorithm='lbfgs', all_possible_states=None,
                                 all_possible_transitions=True, averaging=None,
                                 c=None, c1=None, c2=None,
                                 calibration_candidates=None,
                                 calibration_eta=None,
                                 calibration_max_trials=None,
                                 calibration_rate=None,
                                 calibration_samples=None, delta=None,
                                 epsilon=None, error_sensitive=None,...
                   param_distributions={'c1': <scipy.stats._distn_infrastructure.rv_frozen object at 0x000001867662AC88>,
                                        'c2': <scipy.stats._distn_infrastructure.rv_frozen object at 0x00000186767FA588>},
                   pre_dispatch='2*n_jobs', random_state=None, refit=True,
                   return_t

In [34]:
# crf = rs.best_estimator_
print('best params:', rs.best_params_)
print('best CV score:', rs.best_score_)
print('model size: {:0.2f}M'.format(rs.best_estimator_.size_ / 1000000))

best params: {'c1': 0.7829299127396897, 'c2': 0.057576955210170415}
best CV score: 0.3099099060523515
model size: 0.10M


In [35]:
crf = rs.best_estimator_
y_pred = crf.predict(X_test)
print(metrics.flat_classification_report(
    y_test, y_pred, labels=sorted_labels, digits=3
))

              precision    recall  f1-score   support

       B-OBJ      0.000     0.000     0.000      1051
       I-OBJ      0.000     0.000     0.000       142
  B-PREDFULL      0.678     0.465     0.552       172
  I-PREDFULL      0.670     0.625     0.647       104

   micro avg      0.505     0.099     0.165      1469
   macro avg      0.337     0.273     0.300      1469
weighted avg      0.127     0.099     0.110      1469

